## Example code from kaggle 5days ai agents course

### Setting up

In [1]:
!pip install google-adk


  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached websockets-15.0.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached google_cloud_core-2.5.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached protobuf-5.29.5-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-http to determine which version is compatible with other requirements. This could take a while.
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 1.4 MB/s eta 0:00:00a 0:00:01
Using cached google_cloud_core-2.5.0-py3-none-any.whl (29 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 1.4 MB/s eta 0:00:00a 0:00:01
Using cached protobuf-5.29.5-cp38-abi3-macosx_10_9_univers

In [ ]:
import os
gemini_key = "xxx"

os.environ["GOOGLE_API_KEY"] = gemini_key

In [3]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
# define reties
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,  # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)


# 429 is rate limit error
# 500 is internal server error
# 503 is service unavailable
# 504 is gateway timeout

In [6]:
### First agent
#### a simple agent use google search
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


In [7]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [12]:
response = await runner.run_debug(
    "What is the temperature of Hawaii today?"
)


 ### Continue session: debug_session_id

User > What is the temperature of Hawaii today?
helpful_assistant > The temperature in Hawaii today, December 25, 2025, is around 68°F (20°C) in Hawaii County. In Honolulu, the high is expected to be around 82°F (28°C) with a RealFeel® of 82°F, and the low is around 70°F (21°C) with a RealFeel® of 68°F. Other reports for Honolulu indicate a high near 79°F to 82°F and a low near 71°F to 73°F.


### Multi-agents

In [5]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [6]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [7]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


In [8]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


In [9]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest advancements in open source multimodality lanugage models used in ai agents?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in open source multimodality lanugage models used in ai agents?


ResearchCoordinator > The latest advancements in open-source multimodal language models for AI agents are characterized by enhanced capabilities in understanding and processing diverse data types, a robust and growing open-source ecosystem, sophisticated agentic features, significant performance improvements, and a wide range of real-world applications.

Key developments include:
*   **Enhanced Multimodal Understanding:** Models can now simultaneously process and reason across text, images, audio, and video, leading to more comprehensive comprehension and improved performance. Examples include Grok-1.5V and Qwen3-VL.
*   **Thriving Open-Source Ecosystem:** The availability of open-source models like Aria, LLaVA, GLM-4.6V, and Llama 3.2-Vision is democratizing AI development by providing transparency, customization options, and the ability for self-hosting.
*   **Advanced Agentic Capabilities:** AI agents are gaining the ability to use multimodal tools directly (e.g., GLM-4.6V) and enga


### Example: Blog Post Creation with Sequential Agents
#### Instead of just specific instructions, we can also use a structured approach to guide the agent's behavior.

In [10]:
# Outline Agent: Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",  # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [11]:
# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",  # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [12]:
# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


In [13]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [14]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about how to use open source vision lanugage models for ai agents"
)


 ### Created new session: debug_session_id

User > Write a blog post about how to use open source vision lanugage models for ai agents
OutlineAgent > ## Outline:

**Headline:** Unleash Your AI Agent's Superpowers: Harnessing Open-Source Vision-Language Models

**Introduction Hook:** Imagine an AI agent that can not only understand complex instructions but also "see" and interpret its environment. This isn't science fiction anymore! Open-source Vision-Language Models (VLMs) are revolutionizing how we build intelligent agents, empowering them with a nuanced understanding of both the visual and textual worlds. Ready to unlock this next level of AI capability?

**Main Section 1: What are Vision-Language Models (VLMs) and Why They Matter for Agents**

*   **Bridging the Gap:** Explain how VLMs combine the power of computer vision (understanding images) and natural language processing (understanding text) to create a unified understanding.
*   **Enhanced Agent Capabilities:** Discuss how th

### Example: Parallel Multi-Topic Research

In [15]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [16]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [17]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [18]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [19]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [21]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily AI news on multimodality models and agents on Tech, Health, and Finance"
)


 ### Created new session: debug_session_id

User > Run the daily AI news on multimodality models and agents on Tech, Health, and Finance
FinanceResearcher > **Key Fintech Trends in AI: Multimodality, Agents, and Enhanced Reasoning**

**1. Multimodal AI Models:** These models process diverse data types (text, image, audio, video) simultaneously, leading to more comprehensive insights. Market implications include advanced fraud detection, personalized customer experiences, and richer risk analysis. The future outlook points to more sophisticated models that can understand context and nuance, driving innovation in areas like sentiment analysis and market prediction.

**2. Agentic AI Systems:** AI agents are autonomous programs capable of planning and executing tasks. In finance, this translates to automated trading, proactive customer service, and streamlined compliance. The market is seeing rapid adoption, with implications for efficiency gains and new service models. The future anticip

### Example: Iterative Story Refinement (loop agent)

In [22]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [23]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


In [24]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ exit_loop function created.


In [25]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


In [26]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [27]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a little girl who sees stars and blue sky at the same time"
)


 ### Created new session: debug_session_id

User > Write a short story about a little girl who sees stars and blue skys at the same time
InitialWriterAgent > Lily pressed her nose against the cool glass of the observatory window. Outside, the sky was a brilliant, cloudless blue, the kind that promised a perfect summer day. Yet, through the massive telescope, a different world unfolded. Pinpricks of light, impossibly distant and ancient, glittered against a velvet darkness.

"How can it be both, Mama?" she whispered, her breath fogging the pane.

Her mother smiled, tracing the constellations Lily pointed out. "Because, my sweet star-gazer, the sky is very big. The sun lights up our little piece, but beyond it, the stars have always been there, waiting to be seen." Lily nodded, a universe of wonder opening before her very eyes.
CriticAgent > The story is a charming vignette with a clear premise and a touching moment between mother and daughter. However, to enhance its impact and complet